In [2]:
#meta 4/21/2021 myClassify (multiclass) for LSHTC - Data prep
# task: classify text (multiclass) 
# input: my-input/lshtc3_wikipedia_med_df_train_multilabel.pkl

#task: Multi-category text classification - supervised
#based on aml_0_dataprep.ipynb
#      4/15/2021 DATA PREP 
#      LSHTC3 reduced ds with less labels
#      Important reset index - fast.ai dataloaders don't like a gap in index
#      to avoid -> TypeError: 'float' object is not iterable

#history
#4/21/2021 INPUT DF W/ MULTI-LABELS
#      multi-labels per document
#      requires pandas 1.1.0+ (myTrainBox env nlp_lshtc)

#4/23/2021 REDO INPUT DF W/ MULTI-LABELS
#      Imploded df subset w/ multi-labels
#      Reverted to one field `labels` with strings separated by space (per book Ch.6)
#      Subset Imploded df -> implode labels from each record into docs with multi labels per doc
#      127043 imploded documents x 3 fields docno, text, labels
#      (1 rec = 1 doc -> many labels)
#      confirmed: 139 unique labels


#meta 5/1/2021 Web of Sciences Data - Data prep
#NLP, hierarchical text classification
#src data https://data.mendeley.com/datasets/9rw3vkcfy4/2
#citation: Kowsari, Kamran (2017), “Web of Science”, Mendeley Data, V2, doi: 10.17632/9rw3vkcfy4.2

#input: WOS5736 X and y files
#output: WOS5736_df.pkl with X input and y output (3 levels)

#history
#5/1/2021 READ INPUT FILES AND BUILD A LEARNING SET
#      Prep data - X, y (plus level 1 and level 2 target vars)

In [3]:
%matplotlib inline
#import time as time #to track performance time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,6)
from joblib import load, dump


# LSHTC: Data Prep for Classifying Text

## 0. Load Data
input: a prepared pickle with less labels (~).

In [4]:
df = load('myData/WOS5736_df.pkl') 
print(df.shape)
print(df.columns)
df.head()


(5736, 4)
Index(['X', 'y_l1', 'y_l2', 'y'], dtype='object')


,X,y_l1,y_l2,y
0,Phytoplasmas are insect-vectored bacteria that...,2,2,9
1,"Background: (-)-alpha-Bisabolol, also known as...",2,0,7
2,A universal feature of the replication of posi...,2,0,7
3,"1,2-Dichloropropane (1,2-DCP) and dichlorometh...",2,2,9
4,This paper presents the simulation results of ...,0,1,1


## 1. Data Prep
Tidy data - pick relevant columns

In [7]:
df_tidy = df[['X', 'y']].copy()
df_tidy.columns = ['text', 'labels']
df_tidy['labels'] = df_tidy['labels'].astype('category')
df_tidy.dtypes

text        object
labels    category
dtype: object

ML Expected Format: input features X(matrix) and target variable y(categorical).  
X - column 'text'  
y - columns with 'label' 

In [8]:
#preview ready dataset
df_tidy.head()

,text,labels
0,Phytoplasmas are insect-vectored bacteria that...,9
1,"Background: (-)-alpha-Bisabolol, also known as...",7
2,A universal feature of the replication of posi...,7
3,"1,2-Dichloropropane (1,2-DCP) and dichlorometh...",9
4,This paper presents the simulation results of ...,1


In [ ]:
#reset index - fast.ai dataloaders don't like a gap in index
#to avoid -> TypeError: 'float' object is not iterable
#$note: if need to preserve 'docno', set drop = False
##df_tidy.reset_index(drop=True, inplace=True)
##df_tidy.head(2)


In [10]:
#save file prepped for sentiment analysis
dump(df_tidy, 'myOutput/WOS5736_df_tidy.pkl')

['myOutput/WOS5736_df_tidy.pkl']

In [ ]:
mystop

##### Xtra